# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Longitude,Latitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,san andres,-81.7006,12.5847,27.01,83,20,3.60,CO,1707795784
1,1,bilibino,166.4372,68.0546,-29.90,96,91,1.50,RU,1707795784
2,2,longyearbyen,15.6401,78.2186,-11.09,79,0,0.51,SJ,1707795784
3,3,aasiaat,-52.8699,68.7098,-29.80,97,57,3.12,GL,1707795690
4,4,port isabel,-97.2086,26.0734,16.68,66,0,2.68,US,1707795785


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Longitude",
    "Latitude",
    geo=True,
    tiles="OSM", 
    frame_width=1500,
    frame_height=500,
    size="Humidity",
    color="City"
)

# Display the map plot
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Longitude,Latitude]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
City_ideal = city_data_df.loc[((city_data_df["Max Temp"]<28)&(city_data_df["Max Temp"]>=0)&(city_data_df["Wind Speed"]<12)\
                                & (city_data_df["Cloudiness"]==0))]
# Display sample data
City_ideal.head()

,City_ID,City,Longitude,Latitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
4,4,port isabel,-97.2086,26.0734,16.68,66,0,2.68,US,1707795785
5,5,cockburn town,-71.1419,21.4612,25.27,76,0,8.81,TC,1707795785
7,7,ushuaia,-68.3000,-54.8000,8.81,71,0,1.54,AR,1707795785
9,9,katsuren-haebaru,127.8719,26.3369,23.43,46,0,6.69,JP,1707795785
11,11,guerrero negro,-114.0611,27.9769,16.09,57,0,7.15,MX,1707795786


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
selected_columns = ["City", "Country", "Longitude", "Latitude", "Humidity"]
hotel_df = City_ideal[selected_columns].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df ["Hotel Name"]= ""

# Display sample data
hotel_df.head(15)

,City,Country,Longitude,Latitude,Humidity,Hotel Name
4,port isabel,US,-97.2086,26.0734,66,
5,cockburn town,TC,-71.1419,21.4612,76,
7,ushuaia,AR,-68.3000,-54.8000,71,
9,katsuren-haebaru,JP,127.8719,26.3369,46,
11,guerrero negro,MX,-114.0611,27.9769,57,
28,ad dilam,SA,47.1627,23.9915,84,
44,samho-rodongjagu,KP,127.8711,39.9475,66,
50,lebu,CL,-73.6500,-37.6167,84,
60,maun,BW,23.4167,-19.9833,55,
66,wailua homesteads,US,-159.3780,22.0669,68,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    'apiKey': geoapify_key,
    'categories': 'accommodation.hotel',
}

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    lat = row['Latitude']
    lon = row['Longitude']
    params['filter'] = f'circle:{lon},{lat},{radius}'
    params['bias'] = f'proximity:{lon},{lat}'
    
    base_url = "https://api.geoapify.com/v2/places"
    
    response = requests.get(base_url, params=params)
    response = response.json()
    
    try:
        hotel_name = response["features"][0]["properties"]["name"]
        hotel_df.loc[index, "Hotel Name"] = hotel_name
    except (KeyError, IndexError):
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
    
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display the DataFrame with hotel names
hotel_df.head()

port isabel - nearest hotel: Queen Isabel Inn
cockburn town - nearest hotel: The Salt Raker Inn
ushuaia - nearest hotel: Apart Hotel Aires del Beagle
katsuren-haebaru - nearest hotel: No hotel found
guerrero negro - nearest hotel: Plaza sal paraiso
ad dilam - nearest hotel: No hotel found
samho-rodongjagu - nearest hotel: No hotel found
lebu - nearest hotel: No hotel found
maun - nearest hotel: Center Lodge Conference Center
wailua homesteads - nearest hotel: Hilton Garden Inn Kauai Wailua Bay
emporia - nearest hotel: Budget Host Inn Emporia
comodoro rivadavia - nearest hotel: No hotel found
carnarvon - nearest hotel: No hotel found
general pico - nearest hotel: Euskadi Hotel
pisco - nearest hotel: La Portada
rio gallegos - nearest hotel: Amérian Río Gallegos Apart & Suites
shiraz - nearest hotel: هتل نصیرالملک
qufu - nearest hotel: 曲阜国际饭店
bonthe - nearest hotel: No hotel found
kayes - nearest hotel: No hotel found
ketchikan - nearest hotel: Cape Fox Lodge
fort riley north - nearest ho

,City,Country,Longitude,Latitude,Humidity,Hotel Name
4,port isabel,US,-97.2086,26.0734,66,Queen Isabel Inn
5,cockburn town,TC,-71.1419,21.4612,76,The Salt Raker Inn
7,ushuaia,AR,-68.3000,-54.8000,71,Apart Hotel Aires del Beagle
9,katsuren-haebaru,JP,127.8719,26.3369,46,No hotel found
11,guerrero negro,MX,-114.0611,27.9769,57,Plaza sal paraiso


In [7]:
hotel_df_trimmed = hotel_df[hotel_df["Hotel Name"] != "No hotel found"]
hotel_df_trimmed.head (5)

,City,Country,Longitude,Latitude,Humidity,Hotel Name
4,port isabel,US,-97.2086,26.0734,66,Queen Isabel Inn
5,cockburn town,TC,-71.1419,21.4612,76,The Salt Raker Inn
7,ushuaia,AR,-68.3000,-54.8000,71,Apart Hotel Aires del Beagle
11,guerrero negro,MX,-114.0611,27.9769,57,Plaza sal paraiso
60,maun,BW,23.4167,-19.9833,55,Center Lodge Conference Center


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
# Configure the map plot
map_plot = hotel_df_trimmed.hvplot.points(
    "Longitude",
    "Latitude",
    geo=True,
    tiles="OSM",
    title = "Hotels' locations",
    frame_width=1500,
    frame_height=500,
    size="Humidity",
    color="City",
    hover_cols = ["Hotel Name","Country"]
)
# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Longitude,Latitude]   (City,Humidity,Hotel Name,Country)